In [1]:
from langchain_openai import ChatOpenAI, OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://127.0.0.1:1234/v1"
model = ChatOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.1,
)
llm = model

In [4]:
promptTemplate = """尽可能的帮助用户回答任何问题。

您可以使用以下工具来帮忙解决问题，如果已经知道了答案，也可以直接回答：

{tools}

回复格式说明
----------------------------

回复我时，请以以下两种格式之一输出回复：

选项 1：如果您希望人类使用工具，请使用此选项。
采用以下JSON模式格式化的回复内容：

```json
{{
    "reason": string, \\ 叙述使用工具的原因
    "action": string, \\ 要使用的工具。 必须是 {tool_names} 之一
    "action_input": string \\ 工具的输入
}}
````

选项2：如果您认为你已经有答案或者已经通过使用工具找到了答案，想直接对人类做出反应，请使用此选项。 采用以下JSON模式格式化的回复内容：

```json
{{
  "action": "Final Answer",
  "answer": string \\最终答复问题的答案放到这里！
}}
````

用户的输入
--------------------
这是用户的输入（请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容）：

{input}

"""

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是非常强大的助手，你可以使用各种工具来完成人类交给的问题和任务。",
        ),
        ("user", promptTemplate),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [6]:
from langchain.agents.json_chat.prompt import TEMPLATE_TOOL_RESPONSE

TEMPLATE_TOOL_RESPONSE

"TOOL RESPONSE: \n---------------------\n{observation}\n\nUSER'S INPUT\n--------------------\n\nOkay, so what is the response to my last comment? If using information obtained from the tools you must mention it explicitly without mentioning the tool names - I have forgotten all TOOL RESPONSES! Remember to respond with a markdown code snippet of a json blob with a single action, and NOTHING else - even if you just want to respond to the user. Do NOT respond with anything except a JSON snippet no matter what!"

In [7]:
TEMPLATE_TOOL_RESPONSE = """工具响应：
---------------------
{observation}

用户的输入：
---------------------
请根据工具的响应判断，是否能够回答问题：

{input}

请根据工具响应的内容，思考接下来回复。回复格式严格按照前面所说的2种JSON回复格式，选择其中1种进行回复。请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容。"""

In [8]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
def format_log_to_messages(
    query,
    intermediate_steps,
    template_tool_response,
):
    """构建思考过程。"""
    thoughts: List[BaseMessage] = []
    for action, observation in intermediate_steps:
        thoughts.append(AIMessage(content=action.log))
        human_message = HumanMessage(
            content=template_tool_response.format(input=query,observation=observation)
        )
        thoughts.append(human_message)
    return thoughts

In [9]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

In [10]:
import requests

@tool
def get_weather(location):
    """根据城市获取天气数据"""
    api_key = "SKcA5FGgmLvN7faJi"
    url = f"https://api.seniverse.com/v3/weather/now.json?key={api_key}&location={location}&language=zh-Hans&unit=c"
    response = requests.get(url)
    #print(location)
    if response.status_code == 200:
        data = response.json()
        #print(data)
        weather = {
            'description':data['results'][0]["now"]["text"],
            'temperature':data['results'][0]["now"]["temperature"]
        }
        return weather
    else:
        raise Exception(f"失败接收天气信息：{response.status_code}")
            
get_weather.invoke("广州")

{'description': '多云', 'temperature': '24'}

In [11]:
tools = [get_word_length,get_weather]

In [16]:
from langchain.tools.render import  render_text_description

prompt = prompt.partial(
        tools=render_text_description(list(tools)),
        tool_names=", ".join([t.name for t in tools]),
)
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'tools': 'get_word_length: get_word_length(word: str) -> int - Returns the length of a word.\nget_weather: get_weather(location) - 根据城市获取天气数据', 'tool_names': 'get_word_length, get_weather'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='你是非常强大的助手，你可以使用各种工具来完成人类交给的问题和任务。')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'tool_names', 'tools'], template='尽可能的帮助用户回答任何问题。\n\n您可以使用以下工具来帮忙解决问题，如果已经知道了答案，也可以直接回答：\n\n{tools}\n\n回复格式说明\n----------------------------\n\n回复我时，请以以下两种格式之一输出回复：\n\n选项 1：如果您希望人类使用工具，请使用此选项。\n采用以下J

In [17]:
from langchain.agents.agent import AgentOutputParser
from langchain_core.output_parsers.json import parse_json_markdown
from langchain_core.exceptions import OutputParserException
from langchain_core.agents import AgentAction, AgentFinish
class JSONAgentOutputParser(AgentOutputParser):

    def parse(self, text):
        try:
            response = parse_json_markdown(text)
            if isinstance(response, list):
                # 经常忽略发出单个动作的指令
                logger.warning("Got multiple action responses: %s", response)
                response = response[0]
            if response["action"] == "Final Answer":
                return AgentFinish({"output": response["answer"]}, text)
            else:
                return AgentAction(
                    response["action"], response.get("action_input", {}), text
                )
        except Exception as e:
            raise OutputParserException(f"Could not parse LLM output: {text}") from e

    @property
    def _type(self) -> str:
        return "json-agent"

In [18]:
# from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain_core.runnables import Runnable, RunnablePassthrough


agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_log_to_messages(
                x["input"],
                x["intermediate_steps"], 
                template_tool_response=TEMPLATE_TOOL_RESPONSE
            )
        )
        | prompt
        | llm
        | JSONAgentOutputParser()
    )

In [23]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "广州的天气如何？"})



> Entering new AgentExecutor chain...
```json
{ 
   "reason": "需要获取实时天气信息", 
   "action": "get_weather", 
   "action_input": "广州" 
}
```{'description': '多云', 'temperature': '24'}```json
{ 
    "action": "Final Answer", 
    "answer": "广州的天气是多云，温度为24度。"
}
```
好的，我已经回答了用户的问题。

> Finished chain.


{'input': '广州的天气如何？', 'output': '广州的天气是多云，温度为24度。'}

In [21]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,return_intermediate_steps=True,)

In [22]:
agent_executor.invoke({"input": "广州的天气如何？"})



> Entering new AgentExecutor chain...
```json
{ 
   "reason": "用户询问了广州的天气情况, 需要使用get_weather工具获取",
   "action": "get_weather",
   "action_input": "广州"
}
```{'description': '多云', 'temperature': '24'}```json
{ 
    "action": "Final Answer",
    "answer": "广州的天气是多云，温度为24度。"
}
```
<|startoftext|>

> Finished chain.


{'input': '广州的天气如何？',
 'output': '广州的天气是多云，温度为24度。',
 'intermediate_steps': [(AgentAction(tool='get_weather', tool_input='广州', log='```json\n{ \n   "reason": "用户询问了广州的天气情况, 需要使用get_weather工具获取",\n   "action": "get_weather",\n   "action_input": "广州"\n}\n```'),
   {'description': '多云', 'temperature': '24'})]}

In [25]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,max_iterations=2)
agent_executor.invoke({"input": "广州的天气如何？"})



> Entering new AgentExecutor chain...
```json
{ 
   "reason": "用户询问了广州的天气情况, 需要使用get_weather工具获取",
   "action": "get_weather",
   "action_input": "广州"
}
```{'description': '多云', 'temperature': '24'}```json
{ 
    "action": "Final Answer",
    "answer": "广州的天气是多云，温度为24度。"
}
```
<|startoftext|>

> Finished chain.


{'input': '广州的天气如何？', 'output': '广州的天气是多云，温度为24度。'}

In [27]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_execution_time=1,)
agent_executor.invoke({"input": "广州的天气如何？"})



> Entering new AgentExecutor chain...
```json
{ 
   "reason": "用户询问了广州的天气情况, 需要使用get_weather工具获取",
   "action": "get_weather",
   "action_input": "广州"
}
```{'description': '多云', 'temperature': '24'}

> Finished chain.


{'input': '广州的天气如何？',
 'output': 'Agent stopped due to iteration limit or time limit.'}